# Preparación de la Notebook



In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2020-07-22 17:40:16--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-07-22 17:40:16--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  84.75M   181MB/s    in 0.5s    

2020-07-22 17:40:17 (181 MB/s) - ‘Miniconda3-latest-Linux

In [15]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

#!apt install  openbabel

import rdkit
import pandas as pd
import numpy as np
from rdkit.Chem import PandasTools

!pip install git+https://github.com/autonomio/talos@1.0

In [16]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
%cd drive/My Drive/finalRedes
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/finalRedes'
/content/drive/.shortcut-targets-by-id/1vnOFBVFFehrvQ_ZlIpZVR7M-VhQvq5p-/finalRedes
Descriptores_nr_ar.csv		   tox21_SCAN4_activations
Descriptores_nr_er.csv		   tox21_SCAN4_activations_firstNeuron
Descriptores_test.csv		   tox21_SCAN4_activations_initializers
Descriptores_test_new.csv	   tox21_SCAN4_activations_initializers2
logs				   tox21_SCAN4_relu_dropANDhiddenLayers
Miniconda3-latest-Linux-x86_64.sh  tox21_SCAN4_relu_dropout
nr_ar_activity.txt		   tox21_SCAN4_relu_dropout_bathcNorm
nr-ar.sdf			   tox21_SCAN4_relu_firstNeuron
nr_ar_test_activity.txt		   tox21_SCAN4_relu_kernelReg
nr_er_activity.txt		   tox21_SCAN4_relu_kernelReg_accuracy
nr-er.sdf			   tox21_SCAN4_relu_shapes
nr_er_test_activity.txt		   tox21_SCAN_nr_er_dropout_activations
nr_er_test_new_activity.txt	   tox21_SCAN_nr_er_fi

# Preparación de los datos

Archivo con Descriptores calculados para el efecto toxico NR-ER.

In [17]:
desc_path = 'Descriptores_nr_er.csv' #Este archivo se obtuvo con la notebook tox21_DescriptorsCalc
descriptores = pd.read_csv(desc_path)
descriptores

,Unnamed: 0,abonds,atoms,bonds,cansmi,cansmiNS,dbonds,formula,HBA1,HBA2,HBD,InChI,InChIKey,L5,logP,MP,MR,MW,nF,rotors,s,sbonds,smarts,tbonds,title,TPSA,CHARGE
0,0,11.0,42.0,44.0,0.0,0.0,0.0,0.0,19.0,3.0,2.0,0.0,0.0,0.0,4.7767,133.0267,86.5125,378.312159,6.0,4.0,0.0,33.0,0.0,0.0,0.0,45.15,0
1,1,18.0,69.0,73.0,0.0,0.0,0.0,0.0,39.0,4.0,0.0,0.0,0.0,0.0,6.1944,139.7661,140.9350,457.603800,0.0,7.0,0.0,55.0,0.0,0.0,0.0,30.93,0
2,2,17.0,30.0,32.0,0.0,0.0,0.0,0.0,18.0,7.0,3.0,0.0,0.0,0.0,2.5770,298.4234,73.7772,253.262640,0.0,1.0,0.0,15.0,0.0,0.0,0.0,129.62,0
3,3,12.0,59.0,61.0,0.0,0.0,4.0,0.0,28.0,4.0,2.0,0.0,0.0,0.0,7.0394,202.9346,133.0344,494.475279,6.0,7.0,0.0,45.0,0.0,0.0,0.0,48.78,0
4,4,0.0,92.0,91.0,0.0,0.0,0.0,0.0,64.0,3.0,0.0,0.0,0.0,0.0,7.9826,8.9656,145.4004,474.855700,0.0,25.0,0.0,91.0,0.0,0.0,0.0,27.69,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7692,7692,6.0,21.0,21.0,0.0,0.0,2.0,0.0,12.0,2.0,2.0,0.0,0.0,0.0,1.3850,179.0045,46.8274,170.232100,0.0,2.0,0.0,13.0,0.0,0.0,0.0,80.74,0
7693,7693,0.0,12.0,12.0,0.0,0.0,1.0,0.0,9.0,3.0,2.0,0.0,0.0,0.0,0.1217,82.7273,35.6454,102.158140,0.0,0.0,0.0,11.0,0.0,0.0,0.0,56.15,0
7694,7694,0.0,12.0,12.0,0.0,0.0,1.0,0.0,9.0,3.0,2.0,0.0,0.0,0.0,0.1217,82.7273,35.6454,102.158140,0.0,0.0,0.0,11.0,0.0,0.0,0.0,56.15,0
7695,7695,6.0,32.0,32.0,0.0,0.0,2.0,0.0,19.0,4.0,0.0,0.0,0.0,0.0,4.4449,42.3956,73.9150,291.260621,0.0,7.0,0.0,24.0,0.0,0.0,0.0,115.41,0


Obtenemos el vector de clases segun las actividades

0 : inactivo

1 : activo

In [18]:
#Read target.
actives_f = open("nr_er_activity.txt", "r") #Este archivo se obtuvo de la columna de Actividad del SDF.
activity = []
for x in actives_f:
  activity.append(x[0])
#activity = activity[1:]
activity = [int(i) for i in activity]
y = np.array(activity)
print(y.shape)
actives_f.close()

(7697,)


Vemos la distribucion de Activos e Inactivos

In [19]:
#Visualizar activos vs inactivos: 
activos = sum(y)
inactivos = sum((y == 0)*1)
indices = ['Activos','Inactivos']
print('Proporcion de activos en el dataset:', activos/(activos+inactivos))
activity_df = pd.DataFrame(data = [activos, inactivos], columns = ['NR-ER'], index = indices)
activity_df

Proporcion de activos en el dataset: 0.1217357411978693


,NR-ER
Activos,937
Inactivos,6760


Train - Validation split

In [20]:
#Train-Val Split. 
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(descriptores.values, y, test_size =0.1,random_state = 42, stratify=activity)

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(6927, 27)
(770, 27)
(6927,)
(770,)


# Procesamiento

Normalización

In [21]:
from sklearn.preprocessing import StandardScaler

desc_scaler = StandardScaler()
X_train = desc_scaler.fit_transform(X_train)
Scaled_descriptors = pd.DataFrame(X_train, columns=descriptores.columns)
Scaled_descriptors.head()

,Unnamed: 0,abonds,atoms,bonds,cansmi,cansmiNS,dbonds,formula,HBA1,HBA2,HBD,InChI,InChIKey,L5,logP,MP,MR,MW,nF,rotors,s,sbonds,smarts,tbonds,title,TPSA,CHARGE
0,-0.867422,0.061204,0.681038,0.628320,0.0,0.0,-0.267463,0.0,0.776294,-0.191243,-0.088211,0.0,0.0,0.0,0.631796,-0.323901,0.492766,0.159438,-0.148362,0.746655,0.0,0.686063,0.0,-0.168185,0.0,-0.474732,0.113714
1,-0.218618,0.061204,-0.696640,-0.695395,0.0,0.0,-0.890852,0.0,-0.664650,-0.746224,-0.088211,0.0,0.0,0.0,-0.607300,-0.709506,-0.772663,-0.858691,-0.148362,-0.522930,0.0,-0.697721,0.0,-0.168185,0.0,-0.677815,0.113714
2,0.849975,0.061204,-0.554122,-0.558459,0.0,0.0,-0.267463,0.0,-0.520556,-0.191243,-0.636224,0.0,0.0,0.0,-0.466974,-0.482779,-0.583903,-0.568211,-0.148362,-0.311333,0.0,-0.598879,0.0,-0.168185,0.0,-0.421236,0.113714
3,-0.431886,0.061204,0.015952,-0.010714,0.0,0.0,-0.890852,0.0,0.127869,-0.191243,1.007816,0.0,0.0,0.0,-0.138689,-0.095840,-0.162152,-0.283662,-0.148362,0.323460,0.0,0.043592,0.0,-0.168185,0.0,-0.136819,0.113714
4,-1.397894,-0.992992,0.633532,0.719611,0.0,0.0,0.979316,0.0,0.560152,-0.468734,-0.636224,0.0,0.0,0.0,0.756467,0.446433,0.420815,0.225386,-0.148362,-0.946125,0.0,0.982589,0.0,-0.168185,0.0,-0.444546,0.113714


In [22]:
#Validation Scale:
X_val = desc_scaler.transform(X_val)

Paquetes de procesamiento


In [23]:
import keras
from keras.models import Sequential
from tensorflow.keras import metrics
from tensorflow.keras.metrics import AUC, TruePositives, FalsePositives, FalseNegatives
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization, PReLU, ELU, LeakyReLU
from keras.layers import Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from keras import optimizers, initializers
from tensorflow.keras.optimizers import SGD, Adam
from keras.initializers import glorot_uniform, he_uniform, uniform, Constant
from keras.constraints import maxnorm
from keras.regularizers import l2

import talos
import datetime
from keras.callbacks import TensorBoard
from talos.model.network_shape import network_shape
from talos.model.normalizers import lr_normalizer
from keras.callbacks import EarlyStopping,ReduceLROnPlateau, ModelCheckpoint

## Busqueda de Hiperparametros

Busqueda de Hiperparametros con Talos Scan

In [24]:

def tox21_model(x,y,params): 
    model = Sequential()

    # Input layer
    model.add(Dense(params['first_neuron'], input_shape = (X_train.shape[1],), kernel_initializer='he_uniform', activation = 'tanh'))
    # Input Dropout layer
    if(params['dropout']>0):
        model.add(Dropout(params['dropout']))

    # Hidden layers
    layer_neurons = network_shape(params,1)

    for i in range(params['hidden_layers']):

        # Normalization layer
        if params['batch_norm']==True:
            model.add(BatchNormalization())

        # Dense layer
        model.add(Dense(layer_neurons[i],
                        kernel_initializer=params['kernel_initializer'],
                        kernel_regularizer = params['kernel_regularizer'],
                        activity_regularizer = params['activity_regularizer'],
                        use_bias=True))
        
        # Activation function  
        if params['activation']=='prelu':
            model.add( PReLU(alpha_initializer = Constant(value=params['alpha_initializer']) ))
        elif params['activation']=='elu':
            model.add(ELU(alpha=0.5))
        elif params['activation']=='leaky_relu':
            model.add(LeakyReLU())
        else:
            model.add(Activation(params['activation']))

        # Dropout layer
        if(params['dropout']>0):
            model.add(Dropout(params['dropout']/2))

    # Output Layer
    model.add(Dense(1, kernel_initializer=params['kernel_initializer'], activation = 'sigmoid'))


    # Optimizer
    opt = optimizers.Adam(
                learning_rate=lr_normalizer(params['lr'],Adam),
                beta_1=0.9,
                beta_2=0.999,
                epsilon=1e-7,
                amsgrad=False)
    
    model.compile(loss='binary_crossentropy', optimizer=opt ,metrics=['accuracy'])#,
    
    return model

def tox21_talos(x, y, x_v, y_v, params):

    # Create model
    model = tox21_model(x,y,params)

    print("_____________________________________________________________________")
    for key in params:
        print(f"{key:20}{params[key]}")

    print(model.summary())

    # logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    es = EarlyStopping(monitor='val_loss', verbose=1, patience=25, restore_best_weights = True)
    reduceLR=ReduceLROnPlateau(monitor='val_loss', 
                            factor=0.2, 
                            patience=10, 
                            verbose=1, 
                            min_delta=0, 
                            cooldown=0, min_lr=0)

    history = model.fit(x, y, validation_data = (x_v, y_v),
                        batch_size = params['batch_size'],
                        epochs = params['epochs'],
                        callbacks=[es,reduceLR], #,TensorBoard(logdir, histogram_freq=1)
                        verbose=2)
    
    return history, model

Hiperparametros

Diccionario con los hiperparametros a evaluar. Debido a que la cantidad de hiperparametros es muy grande, se realizo una busqueda por partes para minimizar el tiempo de computo. Esta busqueda conistio en dejar fijos algunos hiperparametros y variar otros, siguiendo un orden:

1. Funciones de activacion e inicializadores

        Funciones de activacion: {Relu, Prelu, LeakyRelu, Helu, Tanh}

        Inicializadores: {Random_uniform, Random_normal, He_uniform, He_normal, Glorot_normal}

2. Neuronas, Hidden Layers, Shapes

        Neuronas: {64, 256, 512, 1024}

        Hidden Layers: {3,4,5}

        Shapes: {Brick, Triangle, Funnel}

3. Regularizacion, Dropout, Batch Normalization
        
        Regularizacion: {None, l1, l2}

        Dropout: {0, 0.2, 0.4}

        Batch Normalization: {True, False}

        

In [25]:
p = {
    'first_neuron': [512],                 # Tamaño de la primera capa
    'dropout': [0,0.2,0.4],                    # Valor de dropout luego de la primera capa
    'activation': ['tanh'],                # Función de activación
    'alpha_initializer': [0.2],             # PRELU alpha
    'batch_size': [256],#, 512],                    # Tamaño de batch
    'epochs': [400],                        # Number of epochs
    'batch_norm': [True,False],                   # Batch normalization layers
    'hidden_layers':[5],                    # Number of hidde layers
    'shapes':['brick'],                     # Shape of hidden layers
    'optimizer': [Adam],                    # Optimizer
    'kernel_regularizer':[None, 'l1,','l2'],            # Kernel (weights) regularization
    #'bias_regularizer':[None],             # Bias (offsets) regularization
    'activity_regularizer':[None],          # Activity (output) regularization
    'kernel_initializer': ['random_normal'], # Kernel (weights) initializer
    'lr': [0.1]                              # Learning rate
}

Talos Scan

Se realiza el scan con cada grupo de hiperparametros y se guardan los resultados en distintos CSVs.

In [ ]:
%matplotlib inline
%reload_ext tensorboard

scan = talos.Scan (x=X_train,
                    y=y_train,
                    model=tox21_talos,
                    experiment_name='tox21_SCAN_nr_er_regularizacion',
                    #fraction_limit=0.004,
                    #round_limit = 200,
                    random_method = 'uniform_mersenne' ,
                    x_val=X_val,
                    y_val=y_val,
                    params=p,
                    print_params=False,) 

#El output de esta funcion se guarda en un archivo csv que permite visualizar los resultados de las metricas para cada combinacion de HP.
    

## Modelo con mejores hiperparametros

Definicion del modelo

Se define el modelo de acuerdo con los hiperparametros que dieron mejor resultado, los cuales se definieron inspeccionando los CSVs creados en la seccion anterior.

In [44]:
# define the model
model = Sequential()

model.add(Dense(512, input_dim = 27, kernel_initializer='he_uniform', activation = 'tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512,kernel_regularizer = 'l2', kernel_initializer='he_uniform', activation ='tanh'))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(512,kernel_regularizer = 'l2',kernel_initializer='he_uniform', activation = 'tanh'))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(512,kernel_regularizer = 'l2',kernel_initializer='he_uniform', activation = 'tanh'))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(512,kernel_regularizer = 'l2',kernel_initializer='he_uniform', activation = 'tanh'))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(1,kernel_initializer='he_uniform', activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               14336     
_________________________________________________________________
batch_normalization_12 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_13 (Batc (None, 512)               2048      
_________________________________________________________________
dense_18 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_14 (Batc (None, 512)              

Optimizador y compile

Dado que las clases se encuentran desbalanceadas (88-12%), se decidio utilizar la metrica AUC, que es el area bajo la curva ROC. 

In [45]:
optimizer = optimizers.Adam(
                learning_rate=lr_normalizer(0.1,Adam),
                beta_1=0.9,
                beta_2=0.999,
                epsilon=1e-7,
                amsgrad=False)

model.compile(loss='binary_crossentropy', optimizer=optimizer ,metrics=[AUC()])

Callbacks

Se utilizara Early Stopping para detener el entrenamiento cuando la "val_loss" deje de mejorar, guardandose los pesos del mejor modelo. Tambien se utilizara el callback ReduceLROnPlateau para disminuir el Learning Rate cuando la "val_loss" no mejore.

In [46]:
es = EarlyStopping(monitor='val_loss', verbose=1, patience=85, restore_best_weights = True)
reduceLR=ReduceLROnPlateau(monitor='val_loss', 
                            factor=0.2, 
                            patience=10, 
                            verbose=1, 
                            min_delta=0, 
                            cooldown=0, min_lr=0)

checkpoint_filepath = 'bestmodel.hdf5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max')

In [47]:
y_train = y_train.reshape(6927,1)
y_val = y_val.reshape(770,1)

Entrenamiento

Se entrena el modelo planteado

In [48]:
hist = model.fit(X_train, y_train, batch_size=256, validation_data=(X_val, y_val), epochs=400, verbose=2, shuffle=True, callbacks=[es, reduceLR])

Train on 6927 samples, validate on 770 samples
Epoch 1/400
 - 1s - loss: 40.9520 - auc_4: 0.5902 - val_loss: 39.8275 - val_auc_4: 0.6323
Epoch 2/400
 - 0s - loss: 39.0129 - auc_4: 0.6527 - val_loss: 37.9825 - val_auc_4: 0.6635
Epoch 3/400
 - 0s - loss: 37.1431 - auc_4: 0.6725 - val_loss: 36.1381 - val_auc_4: 0.6814
Epoch 4/400
 - 0s - loss: 35.2975 - auc_4: 0.6889 - val_loss: 34.3225 - val_auc_4: 0.6967
Epoch 5/400
 - 0s - loss: 33.5179 - auc_4: 0.7008 - val_loss: 32.5856 - val_auc_4: 0.7048
Epoch 6/400
 - 0s - loss: 31.7955 - auc_4: 0.7067 - val_loss: 30.8948 - val_auc_4: 0.7104
Epoch 7/400
 - 0s - loss: 30.1241 - auc_4: 0.7133 - val_loss: 29.2660 - val_auc_4: 0.7166
Epoch 8/400
 - 0s - loss: 28.5317 - auc_4: 0.7186 - val_loss: 27.7092 - val_auc_4: 0.7212
Epoch 9/400
 - 0s - loss: 27.0137 - auc_4: 0.7234 - val_loss: 26.2387 - val_auc_4: 0.7250
Epoch 10/400
 - 0s - loss: 25.5593 - auc_4: 0.7270 - val_loss: 24.8144 - val_auc_4: 0.7290
Epoch 11/400
 - 0s - loss: 24.1663 - auc_4: 0.7315 -

# Dataset de TEST

Archivo con los descriptores del dataset de test

In [31]:
desc = 'Descriptores_test_new.csv' #Este archivo se obtuvo con la notebook tox21_DescriptorsCalc
descriptores_test = pd.read_csv(desc)
descriptores_test

,Unnamed: 0,abonds,atoms,bonds,cansmi,cansmiNS,dbonds,formula,HBA1,HBA2,HBD,InChI,InChIKey,L5,logP,MP,MR,MW,nF,rotors,s,sbonds,smarts,tbonds,title,TPSA,CHARGE
0,0,6.0,95.0,102.0,0.0,0.0,5.0,0.0,58.0,8.0,0.0,0.0,0.0,0.0,5.3452,323.4901,193.7180,610.831900,0.0,6.0,0.0,91.0,0.0,0.0,0.0,72.88,0
1,1,10.0,33.0,36.0,0.0,0.0,1.0,0.0,19.0,11.0,2.0,0.0,0.0,0.0,0.9645,234.5760,76.3992,407.094061,0.0,1.0,0.0,25.0,0.0,0.0,0.0,167.48,-1
2,2,21.0,35.0,39.0,0.0,0.0,1.0,0.0,15.0,3.0,1.0,0.0,0.0,0.0,3.1009,260.0716,87.4097,287.315320,0.0,0.0,0.0,17.0,0.0,0.0,0.0,50.68,0
3,3,17.0,42.0,44.0,0.0,0.0,0.0,0.0,20.0,2.0,1.0,0.0,0.0,0.0,4.4267,127.8196,87.9497,328.355826,2.0,7.0,0.0,27.0,0.0,0.0,0.0,37.91,0
4,4,12.0,41.0,42.0,0.0,0.0,3.0,0.0,22.0,4.0,1.0,0.0,0.0,0.0,4.1642,121.6301,91.2077,351.847720,0.0,7.0,0.0,27.0,0.0,0.0,0.0,71.62,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,642,0.0,13.0,12.0,0.0,0.0,0.0,0.0,10.0,2.0,2.0,0.0,0.0,0.0,0.2986,14.3068,25.6258,92.160020,0.0,2.0,0.0,12.0,0.0,0.0,0.0,59.03,0
643,643,0.0,36.0,36.0,0.0,0.0,0.0,0.0,27.0,6.0,3.0,0.0,0.0,0.0,-2.4309,57.7851,67.1814,219.281340,0.0,6.0,0.0,36.0,0.0,0.0,0.0,70.41,0
644,644,5.0,11.0,11.0,0.0,0.0,1.0,0.0,7.0,4.0,1.0,0.0,0.0,0.0,-0.1273,126.9533,26.4687,116.144960,0.0,0.0,0.0,5.0,0.0,0.0,0.0,78.59,0
645,645,11.0,36.0,38.0,0.0,0.0,1.0,0.0,18.0,4.0,2.0,0.0,0.0,0.0,3.3393,221.4786,86.1255,344.259400,0.0,5.0,0.0,26.0,0.0,0.0,0.0,85.93,0


Obtenemos el vector de clases segun las actividades

0 : inactivo

1 : activo


In [32]:
#Read test targets.
actives_f = open("nr_er_test_new_activity.txt", "r") #Este archivo se obtuvo de la columna de Actividad del SDF.
test_activity = []
for i,x in enumerate(actives_f):
  if(x[0] != 'x'):
    test_activity.append(x[0])
  else:
    descriptores_test = descriptores_test.drop(i,axis =0)
test_activity = [int(i) for i in test_activity]
y_test = np.array(test_activity)
print(y_test.shape)
actives_f.close()

(516,)


Corroboramos la distribucion de clases en test

In [33]:
#Visualizar activos vs inactivos en Test: 
activos = sum(y_test)
inactivos = sum(y_test == 0)
print('Proporcion de activos en el dataset:', activos/(activos+inactivos))
indices = ['Activos','Inactivos']
activity_df = pd.DataFrame(data = [activos, inactivos], columns = ['Test'], index = indices)
activity_df

Proporcion de activos en el dataset: 0.09883720930232558


,Test
Activos,51
Inactivos,465


Normalizacion de los datos de Test

In [34]:
#Test Dataset Scale:
X_test = desc_scaler.transform(descriptores_test)
X_test.shape

(516, 27)

Evaluacion de Test

In [64]:
from sklearn.metrics import auc, roc_auc_score, accuracy_score, roc_curve
#Predict for Test:
predicted = model.predict(X_test)
#Evaluar metrica AUC en test
m = metrics.AUC()
m.update_state(y_test,predicted)
auc_test = m.result().numpy()

print(auc_test)

0.7678895


Para visualizar la matriz de confusión y por ende los valores de VN, FP, FN y VP que permiten una mejor interpretación de los resultados se calcula el umbral óptimo para el cual los valores de Especificidad y Sensibilidad de la curva Roc son mayores.

In [59]:
def Find_Optimal_Cutoff(target, predicted):
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return roc_t['threshold'].values

threshold = Find_Optimal_Cutoff(y_test,predicted)[0]
threshold

0.17864324

Con ese umbral se predicen las clases para el dataset de test y se calcula la matriz de confusión.

In [60]:
from sklearn.metrics import classification_report, confusion_matrix

In [61]:
predicted[predicted<=threshold] = 0
predicted[predicted>threshold] = 1
predicted = predicted.flatten().astype(int)

In [62]:
print(confusion_matrix(y_test,predicted))

[[317 148]
 [ 17  34]]


# Discusión y conclusiones

Para evaluar el roc_auc score obtenido para el dataset de test (0.768) se lo compara con el del equipo ganador de la competencia Tox21 Data Challenge, el unico equipo en utilizar redes neuronales. Nuestro resultado es comparable con el resultado obtenido por el equipo ganador que fue de 0.810. 

Creemos que la principal razón de la diferencia entre los resultados puede deberse a la cantidad de descriptores utilizados para la clasificación. En nuestro caso contamos únicamente con 27 descriptores mientras que en el trabajo del equipo ganador se utilizaron mas de mil descriptores. También puede deberse a la elección de los hiperparámetros.

Es importante comentar que durante la realización del trabajo nos encontramos con el siguiente inconveniente:

Existe una diferencia en la métrica calculada (AUC) durante el entrenamiento y luego del entrenamiento. Esto se evidencio cuando realizamos el calculo del AUC para el dataset de Test, el cual fue considerablemente menor que el calculado durante el entrenamiento para el set de validación. Para verificar esto, recalculamos el AUC para el set de Validación de la misma forma que para Test, obteniendo un resultado distinto al observado durante el entrenamiento.Esta diferencia puede verse al utilizar los metodos predict y evaluate, como se ve a continuacion. 

In [53]:
val_pred = model.predict(X_val)

m = metrics.AUC()
m.update_state(y_val,val_pred)
print('AUC para Val obtenido a partir de las predicciones hechas con predict:', m.result().numpy())

auc_evaluate = model.evaluate(X_val, y_val,batch_size=len(y_val),verbose=0)
print('AUC para Val obtenido con evaluate:', auc_evaluate[1])


AUC para Val obtenido a partir de las predicciones hechas con predict: 0.7619601
AUC para Val obtenido con evaluate: 0.8000931739807129


El valor de AUC no solo cambia despues del entrenamiento sino que esta variacion no es siempre igual, imposibilitando la comparacion entre modelos distintos. Ademas, un resultado en la validacion no garantiza un resultado similar para el test. Hubo casos en los que se obtuvo un score de AUC de 0.93 para la validacion pero al evaluar el set de Test no superaba el 0.7.

Por otro lado, se observo que las metricas de VP, VN, FP y FN daban valores mas altos que la cantidad total de compuestos. Por lo tanto, asumimos que el valor correcto de AUC para comparar el resultado con el de la competencia Tox21 era el obtenido a partir de las predicciones del modelo (utilizando la funcion predict).
